In [36]:
import pandas as pd
import os
import plotly.graph_objects as go

In [37]:
# workingset_sizes = [1, 2, 4, 8, 12, 16]
workingset_sizes = [1, 2, 4, 8]
mig_methods = ['Dtrack']
# mig_methods = ['Vanilla', 'XBZRLE', 'HBFDP_SHA1', 'XBZRLE_HBFDP_SHA1','HBFDP_MD5', 'XBZRLE_HBFDP_MD5']
# mig_workloads = ["Sysbench", 'Quicksort', 'Memcached', 'Oltp']
mig_workloads = ['Memcached']

In [38]:
def fd_plot(workload = False, method = False):
    
    mig_data = {}
    
    if workload :
        for mig_method in mig_methods:
            mig_data[mig_method] = {"data": [], "deviation": []}
            for workingset in workingset_sizes:
                workingset_mb = str(workingset*1024)
                if os.path.exists(f'./data/{mig_method}/{workload}/{workingset_mb}.csv'):
                    data = pd.read_csv(f'./data/{mig_method}/{workload}/{workingset_mb}.csv')
                    mig_data[mig_method]["data"].append(data["Total-Fake-Dirty-Pages"].mean())
                    mig_data[mig_method]["deviation"].append(data["Total-Fake-Dirty-Pages"].std())
        
        data = pd.DataFrame({'RAM Size (GB)': [ str(workingset) for workingset in workingset_sizes]})
    
        count=1
        for mig_method in mig_methods:
            if len(mig_data[mig_method]["data"]) == 6:
                data.insert(column=mig_method, value=mig_data[mig_method]["data"], loc=count)  
                count += 1
        
        
        layout = go.Layout( xaxis=dict(title='RAM Size (GB)'), yaxis=dict(title='Total Fake Dirty Pages'), template='simple_white' )
        
        trace = []
        
        for mig_method in mig_methods:
            name = ""
            for word in mig_method.split("_"):
                name = f'{name} {word}'
                
            if len(mig_data[mig_method]["data"]) == 6:
                trace.append(go.Scatter(x=data['RAM Size (GB)'], y=data[mig_method], name=f'{name} Precopy', mode='lines+markers'))
        
        fig = go.Figure(data=trace, layout=layout)
        
        fig.update_layout(
            # title=f"Fake Dirty Page Count Generated in {workload}",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.5,
            ),
        )
    
        fig.write_image(f"./RAM-Size-vs-Total-Fake-Dirty-Pages.{workload}.png",scale=1, width=1920)
    
        fig.show()
    
    if method :
        for mig_workload in mig_workloads:
            mig_data[mig_workload] = {"data": [], "deviation": []}
            for workingset in workingset_sizes:
                workingset_mb = str(workingset*1024)
                if os.path.exists(f'./data/{method}/{mig_workload}/{workingset_mb}.csv'):
                    data = pd.read_csv(f'./data/{method}/{mig_workload}/{workingset_mb}.csv')
                    mig_data[mig_workload]["data"].append(data["Total-Fake-Dirty-Pages"].mean())
                    mig_data[mig_workload]["deviation"].append(data["Total-Fake-Dirty-Pages"].std())
        
        data = pd.DataFrame({'RAM Size (GB)': [ str(workingset) for workingset in workingset_sizes]})
        
        count=1
        for mig_workload in mig_workloads:
            if len(mig_data[mig_workload]["data"]) == 6:
                data.insert(column=mig_workload, value=mig_data[mig_workload]["data"], loc=count)  
                count += 1
        
        
        layout = go.Layout( xaxis=dict(title='RAM Size (GB)'), yaxis=dict(title='Total Fake Dirty Pages'), template='none' )
        
        trace = []
        
        for mig_workload in mig_workloads:
            name = ""
            for word in mig_workload.split("_"):
                name = f'{name} {word}'
                
            if len(mig_data[mig_workload]["data"]) == 6:
                trace.append(go.Scatter(x=data['RAM Size (GB)'], y=data[mig_workload], name=f'{name}', mode='lines+markers'))
        
        fig = go.Figure(data=trace, layout=layout)
        
        fig.update_layout(
            title=f"Fake Dirty Page Count in {method} Pre-Copy",
            margin=dict(t=45, r=5, l=70, b=45, autoexpand=True),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.3,
            ),
            paper_bgcolor='#F9F9F9',  # Color of the entire figure
            plot_bgcolor='#F9F9F9'
        )
    
        fig.write_image(f"./RAM-Size-vs-Total-Fake-Dirty-Pages.{method}.png",scale=4, width=777, height=500)
    
        fig.show()

In [39]:
# fd_plot(workload = "Sysbench")

In [40]:
fd_plot(method = "Vanilla")